In [1]:
%logstop
%logstart -rtq ~/.logs/ml.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
from static_grader import grader

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [15]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
File ‘./ml-data/providers-train.csv’ already there; not retrieving.

File ‘./ml-data/providers-metadata.csv’ already there; not retrieving.



In [6]:
wget?

Object `wget` not found.


We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [1]:
import numpy as np
import pandas as pd

In [2]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation


In [8]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

In [13]:
fine_totals

0        15259
1            0
2            0
3            0
4            0
         ...  
13887        0
13888        0
13889        0
13890        0
13891    70865
Name: FINE_TOT, Length: 13892, dtype: int64

In [5]:
metadata['Variable']

0            PROVNUM
1           PROVNAME
2            ADDRESS
3               CITY
4              STATE
           ...      
74          FINE_TOT
75        PAYDEN_CNT
76    TOT_PENLTY_CNT
77          LOCATION
78          FILEDATE
Name: Variable, Length: 79, dtype: object

In [9]:
print(metadata[metadata['Variable']=='FINE_CNT'])


    Variable            Label      Description   Format
73  FINE_CNT  Number of Fines  Number of Fines  integer


In [12]:
metadata[metadata['Variable']=='FINE_CNT']

,Variable,Label,Description,Format
73,FINE_CNT,Number of Fines,Number of Fines,integer


In [10]:
def func(groups):
    # default input type dataframeGroupby
    print(groups.shape)
    return

In [11]:
data[data['STATE']=='AX']

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE


In [14]:
data['STATE']

0        AL
1        AL
2        AL
3        AL
4        AL
         ..
13887    TX
13888    TX
13889    TX
13890    TX
13891    TX
Name: STATE, Length: 13892, dtype: object

In [12]:
fine_totals.groupby(data['STATE']).mean().to_dict()

{'AK': 15932.75,
 'AL': 13672.320388349515,
 'AR': 17596.6815920398,
 'AZ': 1512.7222222222222,
 'CA': 8054.977611940299,
 'CO': 22112.54591836735,
 'CT': 8438.121359223302,
 'DC': 27333.933333333334,
 'DE': 24899.0,
 'FL': 16612.338211382114,
 'GA': 29459.975,
 'GU': 0.0,
 'HI': 16133.309523809523,
 'IA': 16565.303571428572,
 'ID': 42741.942028985504,
 'IL': 6634.197226502311,
 'IN': 5626.954545454545,
 'KS': 24420.79109589041,
 'KY': 32656.315384615384,
 'LA': 5611.819277108434,
 'MA': 16722.25925925926,
 'MD': 42806.67661691542,
 'ME': 1275.5869565217392,
 'MI': 21437.39746835443,
 'MN': 3219.326530612245,
 'MO': 7635.160997732426,
 'MS': 7595.4812834224595,
 'MT': 32754.970588235294,
 'NC': 30445.040506329115,
 'ND': 560.171052631579,
 'NE': 8377.755319148937,
 'NH': 260.45588235294116,
 'NJ': 3490.756838905775,
 'NM': 39652.64705882353,
 'NV': 3050.1964285714284,
 'NY': 2213.51526032316,
 'OH': 8214.822977725675,
 'OK': 11812.111111111111,
 'OR': 12365.983606557376,
 'PA': 9216.96

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

In [22]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}
        self.global_average = 0

    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        if not isinstance(X,pd.DataFrame):
            X = pd.DataFrame(X)
        self.group_averages = y.groupby(X[self.grouper]).mean().to_dict()
        self.global_average = y.mean()
        return self

    def predict(self, X):
        # Return a list of predicted penalties based on group of samples in X
        if not isinstance(X,pd.DataFrame):
            X = pd.DataFrame(X)
        return [self.group_averages.get(group,self.global_average) for group in X[self.grouper] ]
        #return [0] * len(X)

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [6]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(memory=None, steps=[('sme', GroupMeanEstimator(grouper='STATE'))],
         verbose=False)

In [23]:
data.sample(2)

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
5331,225260,BLAIRE HOUSE OF MILFORD,20 CLAFLIN STREET,MILFORD,MA,1757,5084731272,170,Worcester,73,...,Medicare and Medicaid,1,1,0,4,1,0,4,2017-06-01,2016-05-06
11844,495370,"BRIDGEWATER HOME , INC.",302 NORTH SECOND STREET,BRIDGEWATER,VA,22812,5408282531,820,Rockingham,127,...,Medicare and Medicaid,12,12,1,72,1,0,72,2017-05-25,2016-04-14


Next we should test that our predict method works.

In [7]:
state_model.predict(data.sample(5))

[18425.87278106509,
 2213.51526032316,
 16565.303571428572,
 7635.160997732426,
 18147.119744058502]

In [16]:
qme = GroupMeanEstimator(grouper='STATE')
qme.fit(data, fine_totals)
qme.group_averages

{'AK': 15932.75,
 'AL': 13672.320388349515,
 'AR': 17596.6815920398,
 'AZ': 1512.7222222222222,
 'CA': 8054.977611940299,
 'CO': 22112.54591836735,
 'CT': 8438.121359223302,
 'DC': 27333.933333333334,
 'DE': 24899.0,
 'FL': 16612.338211382114,
 'GA': 29459.975,
 'GU': 0.0,
 'HI': 16133.309523809523,
 'IA': 16565.303571428572,
 'ID': 42741.942028985504,
 'IL': 6634.197226502311,
 'IN': 5626.954545454545,
 'KS': 24420.79109589041,
 'KY': 32656.315384615384,
 'LA': 5611.819277108434,
 'MA': 16722.25925925926,
 'MD': 42806.67661691542,
 'ME': 1275.5869565217392,
 'MI': 21437.39746835443,
 'MN': 3219.326530612245,
 'MO': 7635.160997732426,
 'MS': 7595.4812834224595,
 'MT': 32754.970588235294,
 'NC': 30445.040506329115,
 'ND': 560.171052631579,
 'NE': 8377.755319148937,
 'NH': 260.45588235294116,
 'NJ': 3490.756838905775,
 'NM': 39652.64705882353,
 'NV': 3050.1964285714284,
 'NY': 2213.51526032316,
 'OH': 8214.822977725675,
 'OK': 11812.111111111111,
 'OR': 12365.983606557376,
 'PA': 9216.96

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [17]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

In [44]:
{'a':[1,2]}

{'a': [1, 2]}

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [18]:
grader.score.ml__state_model(state_model.predict)

Your score:  0.9999999999999999


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [7]:
from sklearn.impute import SimpleImputer
simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        self.replaces = {} # replace missing value of each group

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
        #for colone in self.columns:
         #   if X[colone].dtypes == 'int64' or X[colone].dtypes == 'float64':
          #      self.replaces[colone] = X[colone].mean()
           # if X[colone].dtypes == 'bool':
            #\    self.replaces[colone] = X[colone].describe()['top']
            
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
       
        return X[self.columns]
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('impute',SimpleImputer())
])

In [20]:
np.shape(simple_features.fit_transform(data[:2]))

(2, 7)

In [21]:
data[:4]

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
0,015010,COOSA VALLEY NURSING FACILITY,315 WEST HICKORY STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,85,...,Medicare and Medicaid,7,7,0,36,1,0,36,2017-04-06,2016-05-26
1,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,350,Jackson,50,...,Medicare and Medicaid,5,5,0,44,1,0,44,2017-03-16,2016-02-04
2,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,360,Jefferson,92,...,Medicare and Medicaid,6,6,0,40,1,0,40,2016-10-20,2015-12-30
3,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY P O BOX 97,MC CALLA,AL,35111,2054776161,360,Jefferson,103,...,Medicare and Medicaid,2,2,0,16,1,0,16,2017-03-09,2016-02-11


In [22]:
assert data['RESTOT'].isnull().sum() > 0

**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [23]:
assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data)).any()

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [8]:
from sklearn.linear_model import LogisticRegression
simple_features_model = Pipeline([
    ('simple', simple_features),
    # add your estimator here
    ('est', LogisticRegression())
    
])

In [25]:
simple_features_model.fit(data, fine_counts > 0)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('simple',
                 Pipeline(memory=None,
                          steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('impute',
                                  SimpleImputer(add_indicator=False, copy=True,
                        

In [26]:
fine_counts.value_counts()

0    9612
1    3000
2     892
3     288
4      70
5      24
6       6
Name: FINE_CNT, dtype: int64

In [27]:
fine_counts > 0

0         True
1        False
2        False
3        False
4        False
         ...  
13887    False
13888    False
13889    False
13890    False
13891     True
Name: FINE_CNT, Length: 13892, dtype: bool

In [28]:
def positive_probability(model):
    def predict_proba(X):
        print(model.predict_proba(X)[:, 1])
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

[0.24668334 0.33161747 0.29527525 ... 0.40266376 0.36378874 0.25819655]
Your score:  1.002750512350338


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [9]:
class ColumnTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
    
        return self

    def transform(self, X):
        #if not isinstance(X, pd.DataFrame):
           # X = pd.DataFrame(X)
        X = X.squeeze('columns')
        s = X.value_counts() #data['OWNERSHIP'].value_counts()
        d = {}
        n = s.shape[0]
        for i in range(n):# we built a dictionnary to replace value
            d[s.index[i]] = s[i]
        return X.replace(d)
        


In [10]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
     ('ohe',OneHotEncoder())
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('ohe',OneHotEncoder())
])

categorical_features = FeatureUnion([ 
    ('owner_onehot', owner_onehot),
    ('cert_onehot', cert_onehot)
])

In [31]:
categorical_features.fit_transform(data).dtype

dtype('float64')

In [32]:
data.shape[0]*2

27784

In [33]:
np.isnan([True,True,2]).any()

False

In [34]:
categorical_features.fit_transform(data).dtype

dtype('float64')

In [35]:
categorical_features.fit_transform(data).toarray()

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [36]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data).toarray()).any()

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    # add your estimator here
    #('estimator', RandomForestClassifier(n_estimators=200))
    ('estimator', GridSearchCV(RandomForestClassifier(),
                               param_grid = {'max_depth': range(40,150,10),
                                             'n_estimators': range(100, 200, 25)},
                             cv=5, verbose=1
                              ) )
     
    ])

In [38]:
#gs = GridSearchCV(categorical_features_model,
 #                              param_grid = {'max_depth': range(40,110,10)},
  #                            cv=5, verbose=1
   #                           )

In [39]:
#gs.fit(data, fine_counts > 0)

In [13]:
categorical_features_model.fit(data, fine_counts > 0)

Fitting 5 folds for each of 44 candidates, totalling 220 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 220 out of 220 | elapsed:  1.8min finished


Pipeline(memory=None,
         steps=[('categorical',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('owner_onehot',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(categorical_features=None,
                                                                                categories=None,
                                                                                drop=None,
                                                                                dtype=<class 'numpy.float64'>,
                                                                                handle_unknown='erro

AttributeError: 'Pipeline' object has no attribute 'best_params_'

In [41]:
categorical_features_model.fit(data, fine_counts > 0)

Fitting 5 folds for each of 44 candidates, totalling 220 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 220 out of 220 | elapsed:  1.8min finished


Pipeline(memory=None,
         steps=[('categorical',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('owner_onehot',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(categorical_features=None,
                                                                                categories=None,
                                                                                drop=None,
                                                                                dtype=<class 'numpy.float64'>,
                                                                                handle_unknown='erro

In [42]:
range(100,200,25)

range(100, 200, 25)

In [43]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [12]:
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [13]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

business_model = Pipeline([
    ('features', business_features),
    ('poly', PolynomialFeatures(degree=2)),
    ('svd',TruncatedSVD(n_components=30)),
     #('svd',GridSearchCV(TruncatedSVD(n_components=30),
                           #    param_grid = {'n_components': range(20,40,5)},
                            #   cv=5, verbose=1, scoring='accuracy')),
    
   #('estimator', RandomForestClassifier(n_estimators = 200, min_samples_split = 100))
    ('estimator', GridSearchCV(RandomForestClassifier(min_samples_split = 100 ),
                               param_grid = {'max_depth': range(40,150,10),
                                             'n_estimators': range(100, 200, 25)},
                             cv=5, verbose=1, n_jobs = 1
                              ) )
    # add your estimator here
])

In [46]:
business_model.fit(data, fine_counts > 0)

Fitting 5 folds for each of 44 candidates, totalling 220 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  5.2min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 23.0min
[Parallel(n_jobs=4)]: Done 220 out of 220 | elapsed: 26.3min finished


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('simple',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                   'RESTOT',
                                                                                                   'INHOSP',
                                                                                                   'CCRC_FACIL',
                                                                                                   'SFF',
                                                                                                   'CHOW_LAST_12MOS',
                                      

In [46]:
#business_model.named_steps['svd'].explained_variance_ratio_

In [47]:
#business_model.score(data,fine_counts > 0)

In [48]:
#grader.score.ml__business_model(positive_probability(business_model))

## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [14]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        diff = ((pd.to_datetime(X[self.t2_col]) - 
        pd.to_datetime(X[self.t1_col])).values.astype(float)).reshape(-1,1)
        return diff

In [15]:
import numpy as np
import pandas as pd
np.sum((pd.to_datetime(data['CYCLE_2_SURVEY_DATE']) - 
        pd.to_datetime(data['CYCLE_1_SURVEY_DATE'])).values.astype(float))

-4.658491008e+20

In [15]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In [17]:
time_feature.fit_transform(data)[:5]

array([[-2.72160e+16],
       [-3.50784e+16],
       [-2.54880e+16],
       [-3.38688e+16],
       [-3.32640e+16]])

In [53]:
import datetime
datetime.timedelta(minutes = 1)

datetime.timedelta(seconds=60)

In the cell below we'll collect the cycle 1 survey features.

In [16]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [55]:
cycle_1_features

ColumnSelectTransformer(columns=['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS',
                                 'CYCLE_1_NFROMCOMP', 'CYCLE_1_DEFS_SCORE',
                                 'CYCLE_1_NUMREVIS', 'CYCLE_1_REVISIT_SCORE',
                                 'CYCLE_1_TOTAL_SCORE'])

In [26]:
from sklearn.ensemble import RandomForestRegressor

survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    # add your estimator here
    #('rfr',RandomForestClassifier(n_estimators = 50, max_depth = 5))
    ('poly', PolynomialFeatures(degree=2)),
    ('svd',TruncatedSVD(n_components=30)),
     #('svd',GridSearchCV(TruncatedSVD(n_components=30),
                           #    param_grid = {'n_components': range(20,40,5)},
                            #   cv=5, verbose=1, scoring='accuracy')),
    
   #('rfr', RandomForestRegressor(n_estimators=100, max_depth=20))
    ('estimator', GridSearchCV(RandomForestRegressor(n_estimators=200),
                             param_grid = {'min_samples_leaf': range(20,35,5)
                                            },
                             cv=5, verbose=1
                              ) )
])

In [27]:
survey_model.fit(data, cycle_2_score.astype(int))

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 11.4min finished


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('business',
                                                 FeatureUnion(n_jobs=None,
                                                              transformer_list=[('simple',
                                                                                 Pipeline(memory=None,
                                                                                          steps=[('cst',
                                                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                   'RESTOT',
                                                                                                                                   'INHOSP',
                                               

In [28]:
survey_model.score(data, cycle_2_score.astype(int))

0.12808676685799547

In [30]:
survey_model.predict(data[:5])

array([45.13019259, 40.03368554, 42.31495814, 27.64015731, 36.08449903])

In [29]:
grader.score.ml__survey_model(survey_model.predict)

Your score:  1.1871510789867226


In [37]:
np.array?

array([[1., 2., 3.]])

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*